<a href="https://colab.research.google.com/github/RyanZR/ColabDock-Vina/blob/main/ColabDock_Vina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🍊 ColabDock Vina**
This project is created based on [Lab.06 / IBM3202 – Molecular Docking on AutoDock](https://colab.research.google.com/github/pb3lab/ibm3202/blob/master/tutorials/lab06_docking.ipynb#scrollTo=WutUwTFo2e4j). It provide more dynamic option for visulisation and docking. All credits goes to the author of the mentioned python notebook. 

---

# **`#1` Prepare Environment**
Before starting, we need to install serveral pieces of dependicies and softwares to perform molecular docking. We also need to store data in our Google Drive as data will be lost if browser is closed.
+ [py3Dmol](https://pypi.org/project/py3Dmol/) - *Visualisaion of protein-ligand structure and search grid*
+ [biopython](https://biopython.org/) - *Separation of protein subunits*
+ [condacolab](https://github.com/conda-incubator/condacolab) - *Minimal and colab-friendly installer of conda for software package and environment management*
+ [OpenBabel](https://github.com/openbabel/openbabel) - *Chemical toolbox for file conversion*
+ [MGLtools](https://ccsb.scripps.edu/mgltools/) - *Parameterisation of protein and ligand*
+ [AutoDock Vina](https://vina.scripps.edu/) - *Molecular docking*


In [ ]:
# @title **Install dependencies and softwares**
# @markdown It will take a few seconds. Colab will **restart** after the installation. 
# @markdown Reconnect and execute the next code cell.

# Configure environment 
import os, sys
class Hide:
  def __enter__(self):
    self._original_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
  
  def __exit__(self, exc_type, exc_val, exc_tb):
    sys.stdout.close()
    sys.stdout = self._original_stdout

# Delete sample data
!rm -r /content/sample_data 2>&1 1>/dev/null

# Install dependencies
!pip install py3Dmol 2>&1 1>/dev/null 
!pip install kora 2>&1 1>/dev/null

# Setup AutoDock Vina
!wget -qnc https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz 2>&1 1>/dev/null
!tar xzvf autodock_vina_1_1_2_linux_x86.tgz 2>&1 1>/dev/null
!rm autodock_vina_1_1_2_linux_x86.tgz -f 2>&1 1>/dev/null

# Setup Conda
!pip install -q condacolab 2>&1 1>/dev/null
import condacolab
with Hide():
  condacolab.install_miniconda()
!conda config --set notify_outdated_conda false 2>&1 1>/dev/null

# Install other packages
!conda install -c conda-forge -c bioconda mgltools biopython openbabel zlib --yes 2>&1 1>/dev/null

# Print status
print("> Installation done")
print("> Environment ready for docking")

> Installation done
> Environment ready for docking


In [ ]:
# @title **Import Python libraries**
# @markdown This make Python accessible to different libraries.

# Import libraries
import os
import sys
import shutil
import py3Dmol
import ipywidgets
import kora.install.rdkit
import urllib.request
from Bio.PDB import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact,fixed,IntSlider
from google.colab import drive

# pdb = ".pdb"
# pdbqt = ".pdbqt"
# mol2 = ".mol2"
# smi = ".smiles"
# prot = "_prot"
# bug = "_bug"
# su = "_su"
# exp = "_exp"

class Hide:
  def __enter__(self):
    self._original_stdout = sys.stdout
    sys.stdout = open(os.devnull, "w")
  
  def __exit__(self, exc_type, exc_val, exc_tb):
    sys.stdout.close()
    sys.stdout = self._original_stdout

print("> Import done")

> Import done


In [ ]:
# @title **Import Google Drive**
# @markdown This make Google Drive accessible for data storage.

# Flush and mount GDrive
with Hide():
  drive.flush_and_unmount()
  drive.mount("/content/drive", force_remount=True)
print("> Mounted at /content/drive")

> Mounted at /content/drive


In [ ]:
# @title **Setup alias for Vina**
# @markdown This create an alias for AutoDock Vina.

# Setup alias for Vina
%alias vina /content/autodock_vina_1_1_2_linux_x86/bin/vina
%alias vina_split /content/autodock_vina_1_1_2_linux_x86/bin/vina_split
print("> Alias '%Vina' created")

> Alias '%Vina' created


In [ ]:
# @title **Create folders**
# @markdown This create folders for protein, ligand, experimental results and docking.

# Define path of folder
protein_folder = "/content/protein/"
ligand_folder = "/content/ligand/"
experimental_folder = "/content/experimental/"
docking_folder = "/content/docking/"

folder = [protein_folder,ligand_folder,experimental_folder,docking_folder]

# Create folder if folder have not exists
for f in folder:
  if os.path.exists(Path(f)):
    print("> %s already exists" % f)
  if not os.path.exists(Path(f)):
    os.mkdir(Path(f))
    print("> %s was successfully created" % f)

> /content/protein/ was successfully created
> /content/ligand/ was successfully created
> /content/experimental/ was successfully created
> /content/docking/ was successfully created


---

# **`#2` Prepare Protein**

The first step in docking is to have a structure of a given targer protein. While in some cases a high-quality comparative model will be used, most cases start with an experimentally (X-ray, NMR, cryoEM) solved three-dimensional structure. 

In such cases, a given target protein structure is downloaded from the [Protein Data Bank (PDB)](https://www.rcsb.org/pdb) using a given accession ID.  We can directly download this structure and extract from the PDB.

In [ ]:
# @title **Generate protein PDB file**
# @markdown Enter PDB accession ID to download target protein. This download the
# @markdown the **pdb** format of protein from PubChem.

# Set variables
pdb_id = "7KNX" # @param {type:"string"}
protein = pdb_id

# Download and extract target protein
urllib.request.urlretrieve("http://files.rcsb.org/download/" + protein + ".pdb", protein + ".pdb")
shutil.move("/content/" + protein + ".pdb", protein_folder)
print("> " + protein + ".pdb" + " successfully created in " + protein_folder)

> 7KNX.pdb successfully created in /content/protein/


In [ ]:
# @title **Extract protein**
# @markdown This extract only **amino acids sequence** from pdb file downloaded.   
# @markdown In term of file structure, the **ATOM** lines will be extracted into 
# @markdown a new file. Protein with multiple subunits will also be extracted with 
# @markdown each subsunits as separate files.

# Write lines that contain #string "ATOM" into a new file
with open(protein_folder + protein + "_prot.pdb","w") as g:
  f = open(protein_folder + protein + ".pdb","r")
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
  print("> Protein extracted")
print("> " + protein + "_prot.pdb successfully created in " + protein_folder)

# Separate protein subunits
def separate_protein(molecule):
  chainId = []
  parser = PDBParser()
  io = PDBIO()
  structure  = parser.get_structure("X", protein_folder + molecule)
  numberOfChain = len(list(structure.get_chains()))
  if numberOfChain > 1:
    for chain in structure.get_chains():
      chainId.append(chain.get_id())
      io.set_structure(chain)
      io.save(protein + "_prot_su_" + chain.get_id() + ".pdb")
    for f in os.listdir("/content/"):
      if f.startswith(protein + "_prot_su"):
        shutil.move("/content/" + f, protein_folder)
    print("> " + str(numberOfChain) + " protein subchains detected")
    for n in chainId:
      print("> " + protein + "_prot_su_" + n + ".pdb successfully created in " + protein_folder)
  else:
    pass
separate_protein(protein + "_prot.pdb")

> Protein extracted
> 7KNX_prot.pdb successfully created in /content/protein/
> 4 protein subchains detected
> 7KNX_prot_su_A.pdb successfully created in /content/protein/
> 7KNX_prot_su_B.pdb successfully created in /content/protein/
> 7KNX_prot_su_C.pdb successfully created in /content/protein/
> 7KNX_prot_su_D.pdb successfully created in /content/protein/


In [ ]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for the protein.

def chain_check(input):
  chainId = []
  parser = PDBParser()
  structure  = parser.get_structure("X",input)
  numberOfChain = len(list(structure.get_chains()))
  return numberOfChain

def addBSF(object,add):
  if add:
    object.addStyle({"stick":{"colorscheme":"whiteCarbon"}})

def addLineF(object,add):
  if add:
    object.addStyle({"line":{"style":{"radius":0.1}}})

def showSubunitF(object,input,show,cols):
  if show:
    j = chain_check(input)
    if j > 1:
      for n,m in zip(range(j),cols):
        object.setStyle({"chain":(chr(65+n))},{style:{types:(m if style == "cartoon" else m + "Carbon"),"arrows":True}})
        object.addLabel("Subunit " + chr(65+n),{"fontSize":12,"fontColor":m,"alignment":"topLeft"},{"chain":chr(65+n)})

def showResidF(object,res,highlight):
  if res != "0":
    object.addStyle({"resi": res},{style:{types:(highlight if style == "cartoon" else highlight + "Carbon"),"arrows'":True}})
    object.addResLabels({"resi": res},{"fontSize": 12,"backgroundColor":highlight, "backgroundOpacity":0.7,"inFront":False})

def showLabelF(object,show):
  if show:
    object.addResLabels({"resi": "1-9999"},{"fontSize": 12,"backgroundOpacity": 0.7,"inFront":False})

def showVDWsurfaceF(object,show):
    if show:
      object.addSurface(py3Dmol.VDW,{"opacity":0.5})

# Setup 3D viewer
def view_prot(object,
              input, 
              style = "cartoon",
              types = "color",
              color = "spectrum",
              highlight = "red",
              addBS = False,
              addLine = False,
              showSubunit = False,
              showResid = "0",
              showLabel = False,
              showVDWsurface = False,
              size = (300,300)):
  cols = ["red","orange","yellow","green","blue", "violet","purple","white", "black"] 
  mol1 = open(input,"r").read()
  object.addModel(mol1,"pdb")
  object.setStyle({style:{types:color,"style":"rectangle","arrows":True}})
  addBSF(object,addBS)
  addLineF(object,addLine)
  showSubunitF(object,input,showSubunit,cols)
  showResidF(object,showResid,highlight)
  showLabelF(object,showLabel)
  showVDWsurfaceF(object,showVDWsurface)
  object.setBackgroundColor("0x383838")
  object.zoomTo()
  object.show()

print("> 3D structure viewer created")

> 3D structure viewer created


In [ ]:
# @title **Display 3D protein** { run: "auto" }
# @markdown Enter the protein to be viewed. 
# @markdown This display protein in 3D space with multiple apperance choices.

# Define variable
view = "7KNX_prot_su_A.pdb" #@param {type:"string"}
view_model = "Ribbon Model" #@param ["Line Model","Stick Model", "Ribbon Model"]
model_colour = "spectrum" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black", "spectrum"]
view_residues = "" #@param {type:"string"}
residues_colour = "red" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black"]
style = ""
types = ""
color = ""
show_all_residues = False #@param {type:"boolean"}
show_all_subunits = True #@param {type:"boolean"}
show_VDW_surface = True #@param {type:"boolean"}
add_stick_model = False #@param {type:"boolean"}
add_line_model = False #@param {type:"boolean"}

# Define view_model
if view_model == "Line Model":
  style = "line"
  types = "colorscheme"
if view_model == "Stick Model":
  style = "stick"
  types = "colorscheme"
if view_model == "Ribbon Model":
  style = "cartoon"
  types = "color"
  
# Display 3D structure
@interact
def viewer_one():
  try:
    print("> " + "Showing " + view[:-4] + " ...")
    print("\t")
    res = view_residues.split(",")
    mview = py3Dmol.view()
    view_prot(mview,
              input =  protein_folder + view, 
              style = style,
              types = types,
              color = model_colour,
              highlight = residues_colour,
              addBS = add_stick_model,
              addLine = add_line_model,
              showSubunit = show_all_subunits,
              showResid = res,
              showLabel = show_all_residues,
              showVDWsurface = show_VDW_surface)
  except:
    return None

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [ ]:
# @title **Parameterise protein with Gasteiger charges** 
# @markdown Enter the protein to be parameterised. This add polar hydrogen to the
# @markdown protein and parameterise it with Gasteiger charge using MGLtools.
# @markdown The protein will be converted to **pdbqt** file. In addition, this
# @markdown also fixes the OpenBabel out-of-format conversion bug.

target_protein = "7KNX_prot_su_A.pdb" #@param {type:"string"}
PROTEIN = target_protein[:-4]

# Add polar hydrogen and parameterise with MGLtools
!prepare_receptor4.py -r $protein_folder$PROTEIN".pdb" -o $docking_folder$PROTEIN"_pre.pdbqt" -A hydrogens -U nphs_lps -v > /dev/null 2>&1
shutil.copy(protein_folder + PROTEIN + ".pdb", docking_folder)

# Fix the OpenBabel parsing bug
def fixParse(input,output):
  count = 0
  with open(docking_folder + output, "w") as g:
    f = open(docking_folder + input, "r")
    for line in f:
      if len(list(line)) == 81:
        oldList = list(line)
        del oldList[27]
        line = "".join(map(str, oldList))
        count += 1
      g.write(line)
  print("> Total lines with format error: %s" % count )
  print("> Total lines fixed: %s" % count)

fixParse(PROTEIN + "_pre.pdbqt", PROTEIN + ".pdbqt")
os.remove(docking_folder + PROTEIN + "_pre.pdbqt")
print("> " + PROTEIN + ".pdbqt successfully created in " + docking_folder)


/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> Total lines with format error: 0
> Total lines fixed: 0
> 7KNX_prot_su_A.pdbqt successfully created in /content/docking/


---

# **`#3` Prepare Ligand**

We now need to prepare the ligand that will be used for docking analysis. We will attempt to predict the docking pose onto the binding of target protein.

In [ ]:
# @title **Generate ligand SMILES file**
# @markdown Enter the **canonical** SMILES of ligand.

# Set variables
name = "C26A6" #@param {type:"string"}
smiles = "CC1=NN2C=CC=C(C2=N1)NS(=O)(=O)C3=C(C=CC(=C3)Cl)OC" #@param {type:"string"}
LIGAND = name

# Generate files
with open(ligand_folder + LIGAND + ".smi", 'w') as g:
  g.write(smiles)

print("> " + LIGAND + ".smi successfully created in " + ligand_folder)

> C26A6.smi successfully created in /content/ligand/


In [ ]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for ligand with 100 000 iterations of energy 
# @markdown minimisation via MMFF.
# Convert SMILES to rdkit.Mol with 3D coordinates
def smi_to_conf(smiles): 
  mol = Chem.MolFromSmiles(smiles)
  if mol is not None:
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol, maxIters=100000)
    return mol
  else:
    return None

def addAtomsF(object,add):
  if add:
    object.addPropertyLabels("atom",{},{"fontSize":12,"backgroundOpacity":0.7,"inFront":True})

# View 3D structure
def view_lig(object,mol,addAtoms=False,size=(300, 300)): 
  # assert style in ('line', 'stick', 'sphere', 'carton')
  mblock = Chem.MolToMolBlock(mol)
  object.addModel(mblock, 'mol')
  object.setStyle({"stick":{"style":{"color":"whiteCarbon"}}})
  addAtomsF(object,addAtoms)
  object.setBackgroundColor('0x383838')
  object.zoomTo()
  object.show()

print("> 3D structure viewer created")

> 3D structure viewer created


In [ ]:
# @title **Display 3D ligand** {run: 'auto'}
# @markdown This display molecular dynamic-optimised ligand in 3D space.

show_atoms = True #@param {type:"boolean"}

@interact
def viewer_two():
  try:
    print("> " + "Showing " + LIGAND + " ...")
    print("\t")
    mview = py3Dmol.view()
    conf = smi_to_conf(smiles)
    view_lig(mview,mol = conf,addAtoms = show_atoms)
  except:
    return None

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [ ]:
# @title **Parameterize ligand with Gasteiger charges**
# @markdown This convert **SMILES** to **mol2** file while simultaneously performing 
# @markdown energy minimization using the Generalized Amber Force Field (GAFF) at 
# @markdown 100 000 iterations. After that, this add polar hydrogen to the protein 
# @markdown and parameterise it with Gasteiger charge using MGLtools. The ligand 
# @markdown will be converted to **pdbqt** file. In addition, this also generate
# @markdown a **pdb** file for the ligand.

# Convert from SMILES into a 3D mol2 and pdb format
# Perform energy minimization using the GAFF(Amber)
!obabel $ligand_folder$LIGAND".smi" -O $LIGAND".mol2" --gen3d --best --canonical --minimize --ff GAFF --steps 100000 --sd > /dev/null 2>&1
!obabel $LIGAND".mol2" -O $ligand_folder$LIGAND".pdb" > /dev/null 2>&1

# Add polar hydrogen and parameterise with MGLtools
# Add -z leads to a rigid ligand without any torsions
!prepare_ligand4.py -l $LIGAND".mol2" -o $docking_folder$LIGAND".pdbqt" -U nphs_lps -v > /dev/null 2>&1

shutil.move("/content/" + LIGAND + ".mol2", ligand_folder)
shutil.copy(ligand_folder + LIGAND + ".pdb", docking_folder)
print("> " + LIGAND + ".smi successfully converted to " + LIGAND + ".mol2 and " + LIGAND + ".pdb")
print("> " + LIGAND + ".mol2 successfully converted to " + LIGAND + ".pdbqt in " + docking_folder)

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> C26A6.smi successfully converted to C26A6.mol2 and C26A6.pdb
> C26A6.mol2 successfully converted to C26A6.pdbqt in /content/docking/


---

# **`#4` Set Up Molecular Docking**
It is necessary to define search space for docking on a target protein through the use of grid box. The grid box is usually centered within the binding, active or allosteric site of target protein and its size should be sufficiently enough such that important binding residues are contained inside the box.

In [ ]:
# @title **Setup gridbox** 
# @markdown Click once to create gridbox with 3D protein structure.

#ACKNOWLEDGE: This script is largely based on the one created by Jose Manuel 
#Napoles Duarte, Physics Teacher at the Chemical Sciences Faculty of the 
#Autonomous University of Chihuahua (https://github.com/napoles-uach)

# Define the grid box
def define_grid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({
      'center':{'x':bxi,'y':byi,'z':bzi},
      'dimensions': {'w':bxf,'h':byf,'d':bzf},
      'color':'blue',
      'opacity': 0.6
  })

# Generate 3D protein viewer
def view_dock(object,input,resids):
  mol1 = open(docking_folder + input, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick': {'colorscheme':'whiteCarbon'}})
  object.addResLabels({'resi':resids},{'fontSize': 12,'backgroundOpacity': 0.5,'fixed':0,'inFront':0})

# Combine grid box and protein into a single viewer
def viewer_three(object,input,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10, size=(300,300)):
  # mview = py3Dmol.view()
  define_grid(object,bxi,byi,bzi,bxf,byf,bzf)
  view_dock(object,input,resids)
  object.setBackgroundColor('0x383838')
  object.zoomTo()
  object.show()

print("> Gridbox created")

> Gridbox created


In [ ]:
# @title **Place gridbox at binding site** { run: "auto" }
# @markdown Enter residues without spaces and run the cell again. Place the gridbox
# @markdown as near as the center of binding site 


residues = "255,281" #@param {type:"string"}
X = 57 #@param {type:"slider", min:-100, max:100, step:1}
Y = 52 #@param {type:"slider", min:-100, max:100, step:1}
Z = -52 #@param {type:"slider", min:-100, max:100, step:1}

WIDTH = 16 #@param {type:"slider", min:0, max:30, step:1}
HEIGHT = 22 #@param {type:"slider", min:0, max:30, step:1}
LENGTH = 10 #@param {type:"slider", min:0, max:30, step:1}

@interact
def gridPlacer():
  try:
    print("> " + "Showing " + PROTEIN + " ...")
    print("\t")
    resids = residues.split(",")
    mview = py3Dmol.view()
    viewer_three(mview,PROTEIN + ".pdbqt",resids,X,Y,Z,WIDTH,HEIGHT,LENGTH)
  except:
    return None

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [ ]:
# @title **Generate docking config file**
# @markdown Click to generate config file for AutoDock Vina.

with open(docking_folder + "config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = %s.pdbqt \n" % PROTEIN)
  f.write("ligand = %s.pdbqt \n" % LIGAND)
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES  
  f.write("center_x = %s \n" % X)
  f.write("center_y = %s \n" % Y)
  f.write("center_z = %s \n" % Z)
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = %s \n" % WIDTH)
  f.write("size_y = %s \n" % HEIGHT)
  f.write("size_z = %s \n" % LENGTH)
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

print("> config_singledock file successfully created in /content/docking/")

> config_singledock file successfully created in /content/docking/


---

# **`#5` Perform Molecular Docking**
Autodock Vina will performing the docking stimulation with a progress bar (if running as expected). This simulation should not take longer than 5 min.

In [ ]:
# @title **Run AutoDock Vina**
# @markdown Click to perform molecular docking on targeted protein.

#Changing directory to the single docking folder
os.chdir(docking_folder)

#Executing AutoDock Vina with our configuration file
%vina --config config_singledock --out output.pdbqt --log log.txt

#Exiting the execution directory
os.chdir("/content/")

print("\t> Molecular docking completed")

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 2 CPUs
Reading input ... done.
Set

In [ ]:
# @title **Export different docking pose**
# @markdown This output 9 different docking poses as separate file from AutoDock 
# @markdown Vina.

# Export different docking pose
!obabel -ipdbqt $docking_folder/output.pdbqt -opdb -O $docking_folder$LIGAND"_dock_.pdb" -m > /dev/null 2>&1

result = [f for f in os.listdir(docking_folder) if f.startswith(LIGAND + "_dock")]
number = str(len(result))
print("> " + number + " molecules converted")
print("> " + "The first is " + docking_folder + LIGAND + "_dock_1.pdb")

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> 9 molecules converted
> The first is /content/docking/C26A6_dock_1.pdb


---

# **`#6` Prepare Experimental Ligand**
We can compare the docking poses for verification with the experimentally solved pose for the ligand. 





In [ ]:
# @title **Generate experimental ligand PDB file**
# @markdown Enter the name assigned for the experimental ligand. This extract 
# @markdown all the **experimental ligands** from the pdb file and export it as 
# @markdown separate files if any.

exp_name = "QOV" #@param {type:"string"}

# Extract exp_ligand
with open(experimental_folder + exp_name + ".pdb","w") as g:
  f = open(protein_folder + protein + ".pdb",'r')
  for line in f:
    row = line.split()
    if exp_name in row:
      g.write(line)
  print("> Experimental ligands extracted")
print("> " + exp_name + ".pdb successfully created in " + experimental_folder)

# Separate different experimental pose
def separate_exp(molecule):
  parser = PDBParser()
  io = PDBIO()
  chainId = []
  structure  = parser.get_structure("X", experimental_folder + molecule)
  numberOfChain = len(list(structure.get_chains()))
  if numberOfChain != 1:
    for chain in structure.get_chains():
      chainId.append(chain.get_id())
      io.set_structure(chain)
      io.save(exp_name + "_model_" + chain.get_id() + ".pdb")
    for f in os.listdir("/content/"):
      if f.startswith(exp_name + "_model_"):
        shutil.move("/content/" + f, experimental_folder)
    print("> " + str(numberOfChain) + " models detected")
    for n in chainId:
      print("> " + exp_name + "_model_" + n + ".pdb successfully created in " + experimental_folder) 
  else:
    pass

separate_exp(exp_name + ".pdb")

> Experimental ligands extracted
> QOV.pdb successfully created in /content/experimental/
> 4 models detected
> QOV_model_A.pdb successfully created in /content/experimental/
> QOV_model_B.pdb successfully created in /content/experimental/
> QOV_model_C.pdb successfully created in /content/experimental/
> QOV_model_D.pdb successfully created in /content/experimental/


In [ ]:
# @title **Choose exprimental model**
# @markdown Enter a target model of experimental ligand for comparison.
target_exp = "QOV_model_A.pdb" #@param {type:"string"}
EXP = target_exp[:-4]

# Copy file to docking_folder
shutil.copy(experimental_folder + EXP + ".pdb", docking_folder)
print("> " + EXP + ".pdb successfully created in " + docking_folder)

> QOV_model_A.pdb successfully created in /content/docking/


---

# **`#7` View Docking Results**

The final step is to setup a 3D viewer to display our docking results!

In [ ]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for the protein, docked ligand and the 
# @markdown experimental ligand.

# 
def view_dock_lig(object,input):
  mol = open(input,"r").read()
  object.addModel(mol,"pdb")
  object.setStyle({"model":2},{'stick':{'colorscheme':'greenCarbon'}})

def view_dock_exp(object,input):
  mol = open(input,"r").read()
  object.addModel(mol,"pdb")
  object.setStyle({"model":3},{'stick':{'colorscheme':'grayCarbon'}})

def view_prot_lig_exp(object,
                      inputP, 
                      inputL,
                      inputE,
                      style = "cartoon",
                      types = "color",
                      color = "spectrum",
                      highlight = "red",
                      addBS = False,
                      addLine = False,
                      showSubunit = False,
                      showResid = "0",
                      showLabel = False,
                      showVDWsurface = False,
                      size=(1000,900)):
  cols = ["red","orange","yellow","green","blue", "violet","purple","white", "black"] 
  mol1 = open(inputP,"r").read()
  object.addModel(mol1,"pdb")
  object.setStyle({style:{types:color,"style":"rectangle","arrows":True}})
  addBSF(object,addBS)
  addLineF(object,addLine)
  showSubunitF(object,inputP,showSubunit,cols)
  showResidF(object,showResid,highlight)
  showLabelF(object,showLabel)
  showVDWsurfaceF(object,showVDWsurface)
  view_dock_lig(object,inputL)
  view_dock_exp(object,inputE)
  object.setBackgroundColor("0x383838")
  object.zoomTo()
  object.show()

In [ ]:
# @title **Display docking** {run: 'auto'}
# @markdown Enter the docking pose to be viewed. This display protein, docked 
# @markdown ligand and experimental ligand in 3D space with multiple apperance 
# @markdown choices.

# Define variable
docking_pose = "C26A6_dock_2.pdb" #@param {type:"string"}
view_model = "Ribbon Model" #@param ["Line Model","Stick Model", "Ribbon Model"]
model_colour = "white" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black", "spectrum"]
view_residues = "255,281" #@param {type:"string"}
residues_colour = "red" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black"]
style = ""
types = ""
color = ""
show_all_residues = False #@param {type:"boolean"}
show_all_subunits = True #@param {type:"boolean"}
show_VDW_surface = True #@param {type:"boolean"}
add_stick_model = False #@param {type:"boolean"}
add_line_model = False #@param {type:"boolean"}
dock_lig = docking_pose[:-4]

# Define view_model
if view_model == "Line Model":
  style = "line"
  types = "colorscheme"
if view_model == "Stick Model":
  style = "stick"
  types = "colorscheme"
if view_model == "Ribbon Model":
  style = "cartoon"
  types = "color"
  
# Display 3D structure
@interact
def viewer_four():
  print("> " + "Showing " + PROTEIN + ".pdb ...")
  print("> " + "Showing " + dock_lig + ".pdb ...")
  print("> " + "Showing " + EXP + ".pdb ...")
  print("\t")
  try:
    res = view_residues.split(",")
    mview = py3Dmol.view(1000, 900)
    view_prot_lig_exp(mview,
                inputP =  docking_folder + PROTEIN + ".pdb", 
                inputL =  docking_folder + dock_lig + ".pdb", 
                inputE =  docking_folder + EXP + ".pdb", 
                style = style,
                types = types,
                color = model_colour,
                highlight = residues_colour,
                addBS = add_stick_model,
                addLine = add_line_model,
                showSubunit = show_all_subunits,
                showResid = res,
                showLabel = show_all_residues,
                showVDWsurface = show_VDW_surface)
  except:
    return None

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [ ]:
!mkdir /content/drive/MyDrive/$LIGAND"_"$PROTEIN
!mkdir /content/drive/MyDrive/$LIGAND"_"$PROTEIN/"AutoDock_Vina"
!mkdir /content/drive/MyDrive/$LIGAND"_"$PROTEIN/"GROMACS"
!cp -r /content/docking /content/experimental /content/ligand /content/protein /content/drive/MyDrive/$LIGAND"_"$PROTEIN/"AutoDock_Vina"

print("> Data stored in Google Drive")

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> Data stored in Google Drive
